In [1]:
from collections import defaultdict
from datetime import datetime

In [2]:
countU = defaultdict(int)
countI = defaultdict(int)

with open('../../dataset/lastfm-2k/user_taggedartists-timestamps.dat') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split('\t')
            rev = int(l[0])
            asin = int(l[2])
            
            countU[rev] += 1
            countI[asin] += 1

In [3]:
# 筛时间
T = 201004

# 保证用户在截断时间之前有交互
countUafterT = defaultdict(int)

with open('../../dataset/lastfm-2k/user_taggedartists-timestamps.dat') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split('\t')
            rev = int(l[0])
            asin = int(l[2])
            
            time = datetime.fromtimestamp(int(l[-1]) / 1000).__format__('%Y%m%d')
            
            if int(time[0:6]) >= T:
                countUafterT[rev] += 1

In [4]:
# 筛时间
# 保证用户在截断时间之前与之后有交互
countUbeforeT = defaultdict(int)
# 保证物品在截断时间之前有交互
countIbeforeT = defaultdict(int)

with open('../../dataset/lastfm-2k/user_taggedartists-timestamps.dat') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split('\t')
            rev = int(l[0])
            asin = int(l[2])
            if rev not in countUafterT.keys(): continue
            
            time = datetime.fromtimestamp(int(l[-1]) / 1000).__format__('%Y%m%d')
            
            if int(time[0:6]) < T:
                countIbeforeT[asin] += 1
                countUbeforeT[rev] += 1

In [5]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0

User = dict()
Item = dict()

with open('../../dataset/lastfm-2k/user_taggedartists-timestamps.dat') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split('\t')
            rev = int(l[0])
            asin = int(l[2])
            
            if rev not in countUbeforeT.keys() or asin not in countIbeforeT.keys(): continue
            
            time = datetime.fromtimestamp(int(l[-1]) / 1000).__format__('%Y%m%d')
            
            if rev in usermap: userid = usermap[rev]
            else:
                userid = usernum
                usernum += 1
                usermap[rev] = userid
                User[userid] = []

            if asin in itemmap: itemid = itemmap[asin]
            else:
                itemid = itemnum
                itemnum += 1
                itemmap[asin] = itemid
                Item[itemid] = []


            User[userid].append([itemid, time])
            Item[itemid].append([userid, time])

In [6]:
usernum

640

In [7]:
itemnum

4165

In [8]:
num_int = 0
for k,v in User.items():
    num_int += len(v)

In [9]:
num_int

120975

In [10]:
num_int / (usernum * itemnum) * 100

4.538377851140456

In [11]:
# user部分
User_sorted = dict()
for k,v in User.items():
    User_sorted[k] = sorted(v, key=lambda v: v[1])

In [12]:
User_train = defaultdict(list)
User_test = defaultdict(list)

for k,v in User_sorted.items():
    for i in v:
        time = i[1]
        if int(time[0:6]) < T:
            User_train[k].append(i)
        else:
            User_test[k].append(i)

In [13]:
len(User_train)

640

In [14]:
len(User_test)

575

In [15]:
num_tr = 0
for k,v in User_train.items():
    num_tr += len(v)
num_tr

90136

In [16]:
num_ts = 0
for k,v in User_test.items():
    num_ts += len(v)
num_ts

30839

In [17]:
# 训练集测试集占比
num_ts / (num_tr + num_ts)

0.2549204381070469

In [18]:
# 保证测试集里的所有物品都在训练集里出现过

In [19]:
setI = set()
for k,v in User_train.items():
    for i in v:
        setI.add(i[0])

In [20]:
len(setI)

4165

In [21]:
itemnum

4165

In [22]:
len(setI) == itemnum

True

In [23]:
path = './lastfm/train.txt'
f = open(path, 'w')

for u, i in User_train.items():
    f.write(str(u))
    for n in i:
        f.write(" " + str(n[0]))
    f.write('\n')

f.close()

In [24]:
path = './lastfm/test.txt'
f = open(path, 'w')

for u, i in User_test.items():
    f.write(str(u))
    for n in i:
        f.write(" " + str(n[0]))
    f.write('\n')

f.close()